In [1]:
import requests
import json
import pandas as pd
import bokeh 
from bokeh.plotting import figure
from bokeh.io import output_file, show
import numpy as np
from bokeh.io import curdoc
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.layouts import row, widgetbox
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.layouts import column
from bokeh.models import Select, v
from numpy.random import random, normal, lognormal




In [2]:
# In order to get my data I used an API from NASA. As NASA allows only to scrape 7 days per 7 days
#I first tried all my codes with only a few weeks and added the rest once all sliders and plots were working in JupyterNotebook

key = ''

date_ranges = [
    {'start': '2019-01-01',
    'end': '2019-01-08'},
    {'start': '2019-01-08',
    'end': '2019-01-15'},
    {'start': '2019-01-15',
    'end': '2019-01-22'}, 
    {'start': '2019-01-22',
    'end': '2019-01-29'},
    {'start': '2019-01-29',
    'end': '2019-02-01'},
    {'start': '2019-02-01',
    'end': '2019-02-08'},
    {'start': '2019-02-08',
    'end': '2019-02-15'},
    {'start': '2019-02-15',
    'end': '2019-02-22'},
    {'start': '2019-02-22',
    'end': '2019-02-28'},
    {'start': '2019-02-28',
    'end': '2019-03-07'},
    {'start': '2019-03-07',
    'end': '2019-03-14'},
    {'start': '2019-03-14',
    'end': '2019-03-21'},
    {'start': '2019-03-21',
    'end': '2019-03-28'},
    {'start': '2019-03-28',
    'end': '2019-03-31'},
     {'start': '2019-03-31',
    'end': '2019-04-07'},
    {'start': '2019-04-07',
    'end': '2019-04-14'},
     {'start': '2019-04-14',
    'end': '2019-04-21'},
     {'start': '2019-04-21',
    'end': '2019-04-28'},
     {'start': '2019-04-28',
    'end': '2019-05-05'},
     {'start': '2019-05-05',
    'end': '2019-05-12'},
    {'start': '2019-05-12',
    'end': '2019-05-19'},
    {'start': '2019-05-19',
    'end': '2019-05-26'},
    {'start': '2019-05-26',
    'end': '2019-06-02'},
    {'start': '2019-06-02',
    'end': '2019-06-09'},
    {'start': '2019-06-09',
    'end': '2019-06-16'},
    {'start': '2019-06-16',
    'end': '2019-06-23'},
    {'start': '2019-06-23',
    'end': '2019-06-30'}]

#this is the function I created with some help to reuse the key as much as I wanted.

all_objects = []

for date_range in date_ranges:
    url = 'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}'.format(**{
    'start_date':date_range['start'], 'end_date' : date_range['end'], 'api_key' : key
})
    response = requests.get(url) #this is what I used to filter the columns and keep only the ones I needed.
    raw_data = json.loads(response.text)
    for date in raw_data['near_earth_objects'].keys():
        objects_for_this_day = raw_data['near_earth_objects'][date]
        parsed_objects = []
        for obj in objects_for_this_day:
            parsed_objects.append({
                'estimated_diameter': obj['estimated_diameter'],
                'name': obj['name'],
                'neo_reference_id': obj['neo_reference_id'],
                'absolute_magnitude_h': obj['absolute_magnitude_h'],
                'is_sentry_object': obj['is_sentry_object'],
                'close_approach_day' : obj['close_approach_data'][0]['close_approach_date'],
                'close_approach_full' : obj['close_approach_data'][0]['close_approach_date_full'],
                'relative_velocity_kmh' : obj['close_approach_data'][0]['relative_velocity']['kilometers_per_hour'],
                'miss_distance_lunar' : obj['close_approach_data'][0]['miss_distance']['lunar'],
                'miss_distance_astro' : obj['close_approach_data'][0]['miss_distance']['astronomical'],
                'miss_distance_km' : obj['close_approach_data'][0]['miss_distance']['kilometers'],
                'estimated_diameter_min': float(obj['estimated_diameter']['meters']['estimated_diameter_min']),
                'estimated_diameter_max': float(obj['estimated_diameter']['meters']['estimated_diameter_max']),
            })
        all_objects += parsed_objects

all_obj_dataframe = pd.DataFrame(all_objects)
#estimated diameter_min and _max were changed to floats for creating the column average diameter


In [3]:
all_obj_dataframe

,estimated_diameter,name,neo_reference_id,absolute_magnitude_h,is_sentry_object,close_approach_day,close_approach_full,relative_velocity_kmh,miss_distance_lunar,miss_distance_astro,miss_distance_km,estimated_diameter_min,estimated_diameter_max
0,{'kilometers': {'estimated_diameter_min': 0.88...,162483 (2000 PJ5),2162483,17.400,False,2019-01-08,2019-Jan-08 13:29,132603.7662808978,180.0649385581,0.4628918729,69247638.226150723,880.146521,1968.067451
1,{'kilometers': {'estimated_diameter_min': 0.25...,376864 (2001 TP103),2376864,20.100,False,2019-01-08,2019-Jan-08 18:53,9541.124266855,77.4243774377,0.1990343893,29775120.696030791,253.837029,567.596853
2,{'kilometers': {'estimated_diameter_min': 0.55...,436774 (2012 KY3),2436774,18.400,False,2019-01-08,2019-Jan-08 23:58,55347.376549,177.5726199538,0.4564848842,68289166.363516654,555.334912,1241.766613
3,{'kilometers': {'estimated_diameter_min': 0.24...,513170 (2004 KH15),2513170,20.170,False,2019-01-08,2019-Jan-08 16:58,83140.7020080257,149.9841608884,0.3855633956,57679462.731727372,245.784775,549.591464
4,{'kilometers': {'estimated_diameter_min': 0.10...,(2000 AA6),3025766,22.100,False,2019-01-08,2019-Jan-08 06:01,68857.0657335659,39.7301755586,0.1021341274,15279047.913348638,101.054342,225.964377
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2705,{'kilometers': {'estimated_diameter_min': 0.08...,(2019 KW6),3842875,22.500,False,2019-06-30,2019-Jun-30 12:26,24465.8184275489,80.5438633926,0.2070536334,30974782.532400858,84.053340,187.948982
2706,{'kilometers': {'estimated_diameter_min': 0.05...,(2019 OW),3843350,23.300,False,2019-06-30,2019-Jun-30 08:49,29772.190033583,38.8823772886,0.0999546974,14953009.827534538,58.150704,130.028927
2707,{'kilometers': {'estimated_diameter_min': 0.01...,(2019 VZ),3892274,26.700,False,2019-06-30,2019-Jun-30 19:13,7620.1734128062,89.9823161509,0.2313170081,34604531.706532747,12.149404,27.166893
2708,{'kilometers': {'estimated_diameter_min': 0.00...,(2020 CD3),54000953,31.700,False,2019-06-30,2019-Jun-30 17:33,5603.5412576897,0.7199006716,0.0018506444,276852.460367428,1.214940,2.716689


In [4]:
#create the column average_diameter
all_obj_dataframe["average_diameter"] = all_obj_dataframe [['estimated_diameter_min', 'estimated_diameter_max']].mean(axis=1)



all_obj_dataframe

,estimated_diameter,name,neo_reference_id,absolute_magnitude_h,is_sentry_object,close_approach_day,close_approach_full,relative_velocity_kmh,miss_distance_lunar,miss_distance_astro,miss_distance_km,estimated_diameter_min,estimated_diameter_max,average_diameter
0,{'kilometers': {'estimated_diameter_min': 0.88...,162483 (2000 PJ5),2162483,17.400,False,2019-01-08,2019-Jan-08 13:29,132603.7662808978,180.0649385581,0.4628918729,69247638.226150723,880.146521,1968.067451,1424.106986
1,{'kilometers': {'estimated_diameter_min': 0.25...,376864 (2001 TP103),2376864,20.100,False,2019-01-08,2019-Jan-08 18:53,9541.124266855,77.4243774377,0.1990343893,29775120.696030791,253.837029,567.596853,410.716941
2,{'kilometers': {'estimated_diameter_min': 0.55...,436774 (2012 KY3),2436774,18.400,False,2019-01-08,2019-Jan-08 23:58,55347.376549,177.5726199538,0.4564848842,68289166.363516654,555.334912,1241.766613,898.550762
3,{'kilometers': {'estimated_diameter_min': 0.24...,513170 (2004 KH15),2513170,20.170,False,2019-01-08,2019-Jan-08 16:58,83140.7020080257,149.9841608884,0.3855633956,57679462.731727372,245.784775,549.591464,397.688119
4,{'kilometers': {'estimated_diameter_min': 0.10...,(2000 AA6),3025766,22.100,False,2019-01-08,2019-Jan-08 06:01,68857.0657335659,39.7301755586,0.1021341274,15279047.913348638,101.054342,225.964377,163.509359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2705,{'kilometers': {'estimated_diameter_min': 0.08...,(2019 KW6),3842875,22.500,False,2019-06-30,2019-Jun-30 12:26,24465.8184275489,80.5438633926,0.2070536334,30974782.532400858,84.053340,187.948982,136.001161
2706,{'kilometers': {'estimated_diameter_min': 0.05...,(2019 OW),3843350,23.300,False,2019-06-30,2019-Jun-30 08:49,29772.190033583,38.8823772886,0.0999546974,14953009.827534538,58.150704,130.028927,94.089815
2707,{'kilometers': {'estimated_diameter_min': 0.01...,(2019 VZ),3892274,26.700,False,2019-06-30,2019-Jun-30 19:13,7620.1734128062,89.9823161509,0.2313170081,34604531.706532747,12.149404,27.166893,19.658149
2708,{'kilometers': {'estimated_diameter_min': 0.00...,(2020 CD3),54000953,31.700,False,2019-06-30,2019-Jun-30 17:33,5603.5412576897,0.7199006716,0.0018506444,276852.460367428,1.214940,2.716689,1.965815


In [5]:
#changing to floats to create impact_strength column
all_obj_dataframe[['average_diameter','relative_velocity_kmh']] = all_obj_dataframe[['average_diameter','relative_velocity_kmh']].astype(float)
all_obj_dataframe['impact_strength'] = all_obj_dataframe.average_diameter * all_obj_dataframe.relative_velocity_kmh
all_obj_dataframe

,estimated_diameter,name,neo_reference_id,absolute_magnitude_h,is_sentry_object,close_approach_day,close_approach_full,relative_velocity_kmh,miss_distance_lunar,miss_distance_astro,miss_distance_km,estimated_diameter_min,estimated_diameter_max,average_diameter,impact_strength
0,{'kilometers': {'estimated_diameter_min': 0.88...,162483 (2000 PJ5),2162483,17.400,False,2019-01-08,2019-Jan-08 13:29,132603.766281,180.0649385581,0.4628918729,69247638.226150723,880.146521,1968.067451,1424.106986,1.888419e+08
1,{'kilometers': {'estimated_diameter_min': 0.25...,376864 (2001 TP103),2376864,20.100,False,2019-01-08,2019-Jan-08 18:53,9541.124267,77.4243774377,0.1990343893,29775120.696030791,253.837029,567.596853,410.716941,3.918701e+06
2,{'kilometers': {'estimated_diameter_min': 0.55...,436774 (2012 KY3),2436774,18.400,False,2019-01-08,2019-Jan-08 23:58,55347.376549,177.5726199538,0.4564848842,68289166.363516654,555.334912,1241.766613,898.550762,4.973243e+07
3,{'kilometers': {'estimated_diameter_min': 0.24...,513170 (2004 KH15),2513170,20.170,False,2019-01-08,2019-Jan-08 16:58,83140.702008,149.9841608884,0.3855633956,57679462.731727372,245.784775,549.591464,397.688119,3.306407e+07
4,{'kilometers': {'estimated_diameter_min': 0.10...,(2000 AA6),3025766,22.100,False,2019-01-08,2019-Jan-08 06:01,68857.065734,39.7301755586,0.1021341274,15279047.913348638,101.054342,225.964377,163.509359,1.125877e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2705,{'kilometers': {'estimated_diameter_min': 0.08...,(2019 KW6),3842875,22.500,False,2019-06-30,2019-Jun-30 12:26,24465.818428,80.5438633926,0.2070536334,30974782.532400858,84.053340,187.948982,136.001161,3.327380e+06
2706,{'kilometers': {'estimated_diameter_min': 0.05...,(2019 OW),3843350,23.300,False,2019-06-30,2019-Jun-30 08:49,29772.190034,38.8823772886,0.0999546974,14953009.827534538,58.150704,130.028927,94.089815,2.801260e+06
2707,{'kilometers': {'estimated_diameter_min': 0.01...,(2019 VZ),3892274,26.700,False,2019-06-30,2019-Jun-30 19:13,7620.173413,89.9823161509,0.2313170081,34604531.706532747,12.149404,27.166893,19.658149,1.497985e+05
2708,{'kilometers': {'estimated_diameter_min': 0.00...,(2020 CD3),54000953,31.700,False,2019-06-30,2019-Jun-30 17:33,5603.541258,0.7199006716,0.0018506444,276852.460367428,1.214940,2.716689,1.965815,1.101552e+04


In [6]:
all_obj_dataframe.to_csv("Asteroids_NASA_troiz.csv")